<a href="https://colab.research.google.com/github/ufbfung/pubmed/blob/main/pubmed_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview
This notebook will be used to develop a lightweight, literature assistant that helps with answering research questions by compiling a list of articles from pubmed based on the search keywords provided.

## OpenAI Error codes
|Status code|Error Type|
|--|--|
|400|BadRequestError|
|401|AuthenticationError|
|403|PermissionDeniedError|

# Development

In [22]:
!pip install openai
!pip install beautifulsoup4
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [37]:
import requests
from google.colab import userdata
from openai import OpenAI
from bs4 import BeautifulSoup

def search_pubmed(query_terms, max_articles=20):
    # Step 1: Perform a search and get a list of PubMed IDs
    search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    search_params = {
        'db': 'pubmed',
        'term': ' '.join(query_terms),
        'retmax': max_articles  # Limit the number of articles
    }

    print(query_terms)

    response = requests.get(search_url, params=search_params)
    response_xml = response.text

    # Extract PubMed IDs and filter out articles without abstracts
    article_ids = [id for id in extract_pubmed_ids(response_xml) if has_abstract(id)]

    return article_ids[:max_articles]  # Return only the specified number of article IDs

def has_abstract(article_id):
    # Check if the article has an abstract
    fetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    fetch_params = {
        'db': 'pubmed',
        'id': article_id,
        'retmode': 'xml',
    }

    response = requests.get(fetch_url, params=fetch_params)
    response_xml = response.text

    return '<AbstractText>' in response_xml

def extract_pubmed_ids(xml_response):
    # Parse the XML response and extract PubMed IDs
    article_ids = []
    for line in xml_response.split('\n'):
        if '<Id>' in line:
            article_ids.append(line.replace('<Id>', '').replace('</Id>', ''))
    return article_ids

def retrieve_abstract(article_id):
    # Retrieve abstract for a given PubMed ID
    fetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    fetch_params = {
        'db': 'pubmed',
        'id': article_id,
        'retmode': 'xml',
    }

    response = requests.get(fetch_url, params=fetch_params)
    response_xml = response.text

    # Extract abstract from the XML response
    abstract_start = response_xml.find('<AbstractText>') + len('<AbstractText>')
    abstract_end = response_xml.find('</AbstractText>', abstract_start)

    if abstract_start != -1 and abstract_end != -1:
        abstract = response_xml[abstract_start:abstract_end].strip()
    else:
        abstract = "Abstract not available"

    return abstract

def generate_openai_completion(input_text, research_question):
    #Initialize OpenAI client
    client = OpenAI(api_key=userdata.get('openai'))

    # Response format
    response_format_begin = "Based on the retrieved abstracts, I think the answer to your question is:"

    # Convert input text to a more focused prompt
    prompt = f"Please answer this question: {research_question} using only the list of abstracts retrieved from PubMed here: {input_text}. Format your response by starting with {response_format_begin}. Be sure to answer a question directly, usually a yes or no, followed by supporting reasons you found in the abstracts."

    try:
        # Make a completion request to GPT-3
        response = client.completions.create(
            model="gpt-3.5-turbo-instruct",
            prompt=prompt,
            max_tokens=500
        )

        # Get the generated text from the response
        return response.choices[0].text

    except Exception as e:
        # Handle exceptions and print an error message
        print(f"Error: {e}")
        return None

def extract_pubmed_info(article_id):
    # Extract title and URL from PubMed for a given PubMed ID
    fetch_url = f"https://pubmed.ncbi.nlm.nih.gov/{article_id}"

    response = requests.get(fetch_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract title
    title_tag = soup.find('meta', attrs={'name': 'citation_title'})
    title = title_tag['content'] if title_tag else "Title not available"

    # Construct URL
    article_url = f"https://pubmed.ncbi.nlm.nih.gov/{article_id}/"

    return title, article_url

def generate_and_display_table(article_ids):
    print("\nTable of Retrieved Articles:")
    print("PubMed ID\tTitle\t\t\t\t\t\t\t\t\t\t\t\t\t\tURL")
    for article_id in article_ids:
        title, article_url = extract_pubmed_info(article_id)
        print(f"{article_id}\t\t{title}\t\t\t\t{article_url}")

def summarize_abstracts(article_ids, research_question):
    # Retrieve abstracts and accumulate them
    all_abstracts = ""
    for article_id in article_ids:
        abstract = retrieve_abstract(article_id)
        # print(f"PubMed ID: {article_id}\nAbstract:\n{abstract}\n{'='*30}")
        if abstract is not None:
            all_abstracts += abstract + "\n\n"

    # Send all abstracts and research question to openAI to answer
    summary = generate_openai_completion(all_abstracts, research_question)

    return summary

def main():
    # Get user question
    research_question = input("Enter a question: ").split()

    # Get user input for search terms
    query_terms = input("Enter a list of terms separated by spaces: ").split()

    # Call the function to search PubMed and retrieve abstracts
    article_ids = search_pubmed(query_terms)

    if article_ids:
        # Call the function to summarize the retrieved abstracts
        summary = summarize_abstracts(article_ids, research_question)

        # Print or handle the generated summary as needed
        print(f"Answer:\n{summary}")
    else:
        print("No articles found.")

    # Generate the list of references that were used
    generate_and_display_table(article_ids)

if __name__ == "__main__":
    main()

Enter a question: will ai replace radiologists?
Enter a list of terms separated by spaces: ai radiologists
['ai', 'radiologists']
Answer:


Based on the retrieved abstracts, I think the answer to your question is yes. This is supported by the implementation of an AI software device (qXR) for TB screening using CXR images, which was able to identify additional cases that were not deemed presumptive for TB by radiologists. This highlights the potential of AI in improving the accuracy of TB screening and the need for seamless deployment and integration in resource-limited settings. Additionally, a separate study also found that deep learning-based AI systems show promising performance in assisting radiologists in breast cancer screening.

Table of Retrieved Articles:
PubMed ID	Title														URL
38060461		Implementing a chest X-ray artificial intelligence tool to enhance tuberculosis screening in India: Lessons learned				https://pubmed.ncbi.nlm.nih.gov/38060461/
38043630		Evaluation 